### Click the left side folder buton and Mount Your Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Setting up the workspace

In [2]:
import shutil
import os

shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model1.py' , 'model1.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model2.py' , 'model2.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model3.py' , 'model3.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model4.py' , 'model4.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model5.py' , 'model5.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model6.py' , 'model6.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model7.py' , 'model7.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model8.py' , 'model8.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/model9.py' , 'model9.py')
shutil.copyfile('/content/drive/My Drive/Alzheimer/9 feature extractor pipeline/dataset.py' , 'dataset.py')

os.makedirs('data', exist_ok=True)

shutil.copyfile('/content/drive/My Drive/Alzheimer/micro_frames_tres_127.zip' , './data/micro_frames.zip')
shutil.copyfile('/content/drive/My Drive/Alzheimer/traintestlist.zip' , './data/traintestlist.zip')


!unzip data/micro_frames.zip;
!unzip data/traintestlist.zip;

os.remove("./data/micro_frames.zip")
os.remove('./data/traintestlist.zip')

shutil.move('./micro_frames', './data/micro_frames')
shutil.move('./traintestlist', './data/traintestlist')

Streaming output truncated to the last 5000 lines.
  inflating: micro_frames/667449/77.jpg  
  inflating: micro_frames/667449/78.jpg  
  inflating: micro_frames/667449/79.jpg  
 extracting: micro_frames/667449/8.jpg  
  inflating: micro_frames/667449/80.jpg  
  inflating: micro_frames/667449/81.jpg  
  inflating: micro_frames/667449/82.jpg  
  inflating: micro_frames/667449/83.jpg  
  inflating: micro_frames/667449/84.jpg  
  inflating: micro_frames/667449/85.jpg  
  inflating: micro_frames/667449/86.jpg  
  inflating: micro_frames/667449/87.jpg  
  inflating: micro_frames/667449/9.jpg  
   creating: micro_frames/667564/
  inflating: micro_frames/667564/0.jpg  
  inflating: micro_frames/667564/1.jpg  
  inflating: micro_frames/667564/10.jpg  
  inflating: micro_frames/667564/11.jpg  
  inflating: micro_frames/667564/12.jpg  
  inflating: micro_frames/667564/13.jpg  
  inflating: micro_frames/667564/14.jpg  
  inflating: micro_frames/667564/15.jpg  
  inflating: micro_frames/667564/16.j

'./data/traintestlist'

# Balancebatch

In [1]:
import torch
import torchvision
import torch.utils.data
import random


#Balance Batch generator for eighteen classes
class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
        self.dataset = {}
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = self._get_label(dataset, idx)
            if label not in self.dataset:
                self.dataset[label] = []
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
    
        self.keys = list(self.dataset.keys())
        self.currentkey = 0

    def __iter__(self):
        while len(self.dataset[self.keys[self.currentkey]]) > 0:
            yield self.dataset[self.keys[self.currentkey]].pop()
            self.currentkey = (self.currentkey + 1) % len(self.keys)

    
    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            (image_sequence, target) = dataset.__getitem__(idx)
            return target

    def __len__(self):
        return self.balanced_max*len(self.keys)

# Training



*   Look for the comments in the import section
*   Look for the comments in the begining of the code where
all the paths are declaired, I have added short comments to
understand the usefulness of each variable
*    One note I want to make about batch balance is that more often than not it causes training bias, so far what I have I have seen a training bias but without balance batch the model fails to learn completely. So remember this fact while training and if you want to remove batch balance then delete the "sampler" parameter in the "train_dataloader" variable and comment out the last line of the training cell



In [2]:
import torch
import sys
import numpy as np
import itertools
from model1 import * #depending on which model you're training import model file accordingly 
from dataset import *
from torch.utils.data import DataLoader
from torch.autograd import Variable


import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import matthews_corrcoef as mcc


torch.manual_seed(0)

MCC_SCORE = 0


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]

    del(y_pred)
    del(y_true)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(20, 20))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


def test_model(epoch):
    """ Evaluate the model on the test set """
    print("")
    #defining numpy array to store the true labels and predicted labels
    y_true = np.array([])
    y_pred = np.array([])

    # Preparing the model for evaluation
    model.eval()
    test_metrics = {"loss": [], "acc": []}
    for batch_i, (X, y) in enumerate(test_dataloader):
        image_sequences = Variable(X.to(device), requires_grad=False)
        labels = Variable(y, requires_grad=False).to(device)
        y_true = np.append(y_true, labels.cpu().numpy())
        with torch.no_grad():
            

            # Get sequence predictions
            predictions = model(image_sequences)

            acc = torch.tensor([0 if i<=0.5 else 1 for i in predictions]).to(device)

            y_pred = np.append(y_pred, acc.detach().cpu().numpy())
        # Compute metrics
        
        acc = 100 * (acc.detach() == labels).cpu().numpy().mean()
        labels = labels.view(-1,1)
        loss =   cls_criterion(predictions.float(), labels.float()).item()
        # Keep track of loss and accuracy
        test_metrics["loss"].append(loss)
        test_metrics["acc"].append(acc)
        # Log test performance
        sys.stdout.write(
            "Testing -- [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
            % (
                batch_i,
                len(test_dataloader),
                loss,
                np.mean(test_metrics["loss"]),
                acc,
                np.mean(test_metrics["acc"]),
            )
        )
        del(X)
        del(y)
        del(image_sequences)
        del(labels)
        del(predictions)
        del(loss)

    final_acc=round(np.mean(test_metrics["acc"]),3)

    

    # Save model checkpoint
    
    # Using the learning rate scheduler while monitoring Loss and printing the learning rate after every epoch
    for param_group in optimizer.param_groups:
        print("\nCurrent Learning Rate is : " + str(param_group['lr']))
    model.train()
    print("")
    # Getting mcc score for evaluation and plotting the confusion matrix and saving that matrix
    mcc_score = round(mcc(y_true.astype(int), y_pred.astype(int)), 5)


    mcc_score_str = "MCC Score: " + str(mcc_score) + "\n\n"

    plot_title = mcc_score_str + "Confusion matrix, Without Normalization\n"

    
    class_names = ['Non-stalled', 'Stalled']
    class_names = np.array(class_names)
    plot_confusion_matrix(y_true.astype(int), y_pred.astype(int), classes=class_names, title=plot_title)

    # Save model checkpoint
    global MCC_SCORE

    if MCC_SCORE < mcc_score:
        MCC_SCORE = mcc_score
        os.makedirs("model_checkpoints", exist_ok=True)
        torch.save(model.state_dict(), f"model_checkpoints/model_{model_flag}_epoch_{epoch}_IS_{img_dim}_acc_{final_acc}_mcc_{mcc_score}.pth")

    os.makedirs("confusion_matrix", exist_ok=True)
    plt.savefig(f"confusion_matrix/epoch_{epoch}_acc_{final_acc}_mcc_{mcc_score}.png")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_path = 'data/micro_frames' #path to the dataset (frame) folder
split_path = 'data/traintestlist' # path to the csv folds folder
split_number = 0 #which number of folds to use for training options are one of these four {0,1,2,3}
num_epochs = 150 #number of epochs to run
batch_size = 32 #batch_size per interation
img_dim = 224 # image dimention , this should be same for all the 9 feature extractor training at least for now
channels = 1 # number of channels in a image, it should always be kept 1
latent_dim = 512 # latent dimention , this is also should be same for all the 9 feature extractor training at least for now
hidden_dim = 1024 # try to give value above latent dim and two to the power
model_flag = 0 #this is to specify which image to take according to the feature extractor model value can be between 0-8
checkpoint_model = '' # pretrained weight file if you have any

image_shape = (channels, img_dim, img_dim)



# Define training set
train_dataset = Dataset(
    dataset_path=dataset_path,
    split_path=split_path,
    model_flag = model_flag, 
    split_number=split_number,
    input_shape=image_shape,
    training=True,
)
train_dataloader = DataLoader(train_dataset, sampler=BalancedBatchSampler(train_dataset),batch_size= batch_size, shuffle=False, num_workers=4)
# Define test set
test_dataset = Dataset(
    dataset_path=dataset_path,
    split_path=split_path,
    model_flag = 0, 
    split_number=split_number,
    input_shape=image_shape,
    training=False,
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#cls_weight = torch.tensor([1.6994328922495274])

# Classification criterion
cls_criterion = nn.BCEWithLogitsLoss().to(device)
# Define network
model = ConvLSTM(
    num_classes=2,
    latent_dim=latent_dim,
    hidden_dim=hidden_dim
)
#primarily setting the attention to False 

model = model.to(device)

# model.half()  
# for layer in model.modules():
#   if isinstance(layer, nn.BatchNorm2d):
#     layer.float()

# Add weights from checkpoint model if specified
if checkpoint_model:
    model.load_state_dict(torch.load(checkpoint_model))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


#Stating the epoch
for epoch in range(num_epochs):
    epoch_metrics = {"loss": [], "acc": []}
    print(f"--- Epoch {epoch}---")
    for batch_i, (X, y) in enumerate(train_dataloader):
        if X.size(0) == 1:
            continue


        image_sequences = Variable(X.to(device), requires_grad=True)
        #image_sequences = image_sequences.half()
        labels = Variable(y.to(device), requires_grad=False)
        

        optimizer.zero_grad()

        
        # Get sequence predictions
        predictions = model(image_sequences)


        
        # Compute metrics
        acc = torch.tensor([0 if i<=0.5 else 1 for i in predictions]).to(device)
        acc = 100 * (acc.detach() == labels).cpu().numpy().mean()
        labels = labels.view(-1, 1)
        loss = cls_criterion.forward(predictions.float(), labels.float())
        

        

        loss.backward()
        optimizer.step()

        # Keep track of epoch metrics
        epoch_metrics["loss"].append(loss.item())
        epoch_metrics["acc"].append(acc)

        # Determine approximate time left
        batches_done = epoch * len(train_dataloader) + batch_i
        batches_left = num_epochs * len(train_dataloader) - batches_done

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
            % (
                epoch,
                num_epochs,
                batch_i,
                len(train_dataloader),
                loss.item(),
                np.mean(epoch_metrics["loss"]),
                acc,
                np.mean(epoch_metrics["acc"]),
            )
        )
        del(X)
        del(y)
        del(image_sequences)
        del(labels)
        del(predictions)
        del(loss)


        # Empty cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Evaluate the model on the test set
    test_model(epoch)
    train_dataloader = DataLoader(train_dataset, sampler=BalancedBatchSampler(train_dataset),batch_size= batch_size, shuffle=False, num_workers=4)




Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



--- Epoch 0---
[Epoch 0/150] [Batch 2/80] [Loss: 0.734387 (0.738799), Acc: 50.00% (46.88%)]

KeyboardInterrupt: ignored

In [ ]:
!zip -r /content/model_checkpoints.zip /content/model_checkpoints

print('Model checkpoint done')

!zip -r /content/confusion_matrix.zip /content/confusion_matrix



  adding: content/model_checkpoints/ (stored 0%)
  adding: content/model_checkpoints/ConvLSTM_18for_image_size_112_best_69.0_mcc_0.23757.pth (deflated 8%)
  adding: content/model_checkpoints/ConvLSTM_8for_image_size_112_best_71.167_mcc_0.24971.pth (deflated 8%)
  adding: content/model_checkpoints/.ipynb_checkpoints/ (stored 0%)
Model checkpoint done
  adding: content/confusion_matrix/ (stored 0%)
  adding: content/confusion_matrix/score18with_accuracy67.833_mcc_0.1762.png (deflated 39%)
  adding: content/confusion_matrix/score17with_accuracy67.833_mcc_0.1762.png (deflated 39%)
  adding: content/confusion_matrix/score7with_accuracy68.333_mcc_0.18793.png (deflated 38%)
  adding: content/confusion_matrix/score6with_accuracy68.667_mcc_0.19407.png (deflated 38%)
  adding: content/confusion_matrix/score4with_accuracy69.5_mcc_0.20733.png (deflated 38%)
  adding: content/confusion_matrix/score5with_accuracy69.333_mcc_0.2091.png (deflated 38%)
  adding: content/confusion_matrix/score16with_accu

In [ ]:
import shutil

shutil.rmtree('/content/confusion_matrix')
shutil.rmtree('/content/model_checkpoints')


In [ ]:
import torch

z1=torch.randn((2,1024))
z2=torch.randn((2,1024))
z3=torch.randn((2,1024))
z4=torch.randn((2,1024))
z5=torch.randn((2,1024))
z6=torch.randn((2,1024))
z7=torch.randn((2,1024))
z8=torch.randn((2,1024))
z9=torch.randn((2,1024))
i=torch.stack((z1,z2,z3,z4,z5,z6,z7,z8,z9))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

m = nn.Sequential(
    nn.Linear(1024,1),
    nn.Softmax(0)
)

o = m(i)

print(i.shape)
print(o.shape)

o


torch.Size([9, 2, 1024])
torch.Size([9, 2, 1])


tensor([[[0.1279],
         [0.1271]],

        [[0.0575],
         [0.1738]],

        [[0.2523],
         [0.1432]],

        [[0.1040],
         [0.0616]],

        [[0.1012],
         [0.0346]],

        [[0.1365],
         [0.1329]],

        [[0.0708],
         [0.1984]],

        [[0.0810],
         [0.0765]],

        [[0.0687],
         [0.0519]]], grad_fn=<SoftmaxBackward>)

In [ ]:
z1

tensor([[ 0.0186, -0.5369,  0.9274,  ..., -0.6349,  1.5856,  0.5842],
        [-0.8807, -0.0580,  1.1085,  ..., -0.4956,  0.2532, -0.7793]])

In [ ]:
o[0]

tensor([[0.1279],
        [0.1271]], grad_fn=<SelectBackward>)

In [ ]:
zi = torch.mul(z1, o[0])
z = [z1,z2,z3]
z_ = torch.cat(z, 1)

In [ ]:
z_.shape

torch.Size([2, 3072])